In [ ]:
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
# import dwf

from ctypes import *

%load_ext autoreload
%autoreload 2

In [ ]:
from chip_systems import AD2

In [ ]:
interface = AD2.AD2_I2C(voltage=0, pin=0)
# Green - DIO0 - SCL
# Yellow - DIO1 - SDA

In [ ]:
# Check BME680 WHOAMI - should be 97
interface.ReadRegister(0b1110110, 0xD0, bytes_to_read=1)

In [ ]:
# Register Map
reg_map = {"Status": [0x00, 0, 16, False],
"Br": [0x00, 15, 1, False],
"Smx": [0x00, 14, 1, False],
"Tmx": [0x00, 13, 1, False],
"Vmx": [0x00, 12, 1, False],
"Bi": [0x00, 11, 1, False],
"Smn": [0x00, 10, 1, False],
"Tmn": [0x00, 9, 1, False],
"Vmn": [0x00, 8, 1, False],
"dSOCi": [0x00, 7, 1, False],
"Imx": [0x00, 6, 1, False],
"Bst": [0x00, 3, 1, False],
"Imn": [0x00, 2, 1, False],
"POR": [0x00, 1, 1, False],

"Current": [0x0A, 0, 16, False],
"AvgCurrent": [0x0B, 0, 16, False],

"Temperature": [0x08, 0, 16, False],
"AvgTemperature": [0x16, 0, 16, False],

"VAlrtTh": [0x01, 0, 16, False],

"IChgTerm": [0x1E, 0, 16, False],
"DesignCap": [0x18, 0, 16, False],

"VEmpty": [0x03A, 0, 16, False],
"VE": [0x03A, 7, 9, False],
"VR": [0x01, 0, 7, False],
"VAlrtTh": [0x01, 0, 16, False],

"ModelCFG": [0xDB, 0, 16, False],
"Refresh": [0xDB, 15, 1, False],
"R100": [0xDB, 13, 1, False],
"VChg": [0xDB, 10, 1, False],
"ModelID": [0xDB, 4, 4, False],
"CSEL": [0xDB, 2, 1, False],

"Config": [0x1D, 0, 16, False],
"TSel": [0x1D, 15, 1, False],
"SS": [0x1D, 14, 1, False],
"TS": [0x1D, 13, 1, False],
"VS": [0x1D, 12, 1, False],
"IS": [0x1D, 11, 1, False],
"THSH": [0x1D, 10, 1, False],
"Ten": [0x1D, 9, 1, False],
"Tex": [0x1D, 8, 1, False],
"SHDN": [0x1D, 7, 1, False],
"COMMSH": [0x1D, 6, 1, False],
"ETHRM": [0x1D, 4, 1, False],
"FTHRM": [0x1D, 3, 1, False],
"Aen": [0x1D, 2, 1, False],
"Bei": [0x1D, 1, 1, False],
"Ber": [0x1D, 0, 1, False],

"Config2": [0xBB, 0, 16, False],
"AltRateEn": [0xBB, 13, 1, False],
"DPEn": [0xBB, 12, 1, False],
"POWR": [0xBB, 8, 4, False],
"dSOCen": [0xBB, 7, 1, False],
"TAIrtEn": [0xBB, 6, 1, False],
"LDMdl": [0xBB, 5, 1, False],
"DRCfg": [0xBB, 2, 2, False],
"CPMode": [0xBB, 1, 1, False],

"RepCap": [0x05, 0, 16, False],
"RepSOC": [0x06, 0, 16, False],
"FullCapRep": [0x10, 0, 16, False],
"TTE": [0x11, 0, 16, False],
"TTF": [0x20, 0, 16, False],

"VCell": [0x09, 0, 16, False],
"AvgVCell": [0x19, 0, 16, False],
"MaxMinVolt": [0xFF, 0, 16, False],
"MaxVCELL": [0xFF, 8, 8, False],
"MinVCELL": [0xFF, 0, 8, False],

"FStat": [0x3D, 0, 16, False],
"HibCfg": [0xBA, 0, 16, False]}

In [ ]:
def ReadMax(register):

    if type(register) == str:
        register = reg_map[register][0]

    lsb, msb = interface.ReadRegister(0x36, register, bytes_to_read=2)
    decimal = (msb << 8) + lsb
    binary = format(decimal, '016b')
    binary = binary[:8] + ', ' + binary[8:]
    return binary, decimal

In [ ]:
def WriteMax(register, value):

    if type(register) == str:
        register = reg_map[register][0]

    interface.WriteRegister16(0x36, register, value)

In [ ]:
DesignCap = 2000  # mAh
IchgTerm = 10  # mA
VEmpty = 3.0   # Minimum Voltage
VRecovery = 3.7
ChargeVoltage = 4.2

sec_per_lsb = 5.625
per_per_lsb = 1.0 / 256.0
mah_per_lsb = 0.5
ua_per_lsb = 156.25
v_per_lsb =  78.125 * 1e-6
v_empty_per_lsb = 10.0 * 1e-3  # V / LSB
v_recovery_per_lsb = 40.0 * 1e-3  # V / LSB
i_term_per_lsb = 1.0 / 6.4  # mA / LSB
degc_per_lsb = 1.0 / 256.0

# Scale factor for VEmpty
VE = int(VEmpty / v_empty_per_lsb)  # Empty Voltage - resolution is 10mV / LSB
VR = VRecovery / v_recovery_per_lsb  # Recovery Voltage (voltage for clearing empty detection) - resolution is 40mV / LSB
VEmpty = int((VE << 7) + VR)

# Scale factor for IchgTerm
IchgTerm = int(IchgTerm / i_term_per_lsb)

# Make sure POR is 1
if (ReadMax("POR")[1]):

    while(ReadMax("FStat")[1] & 1):
        time.sleep(10 / 1e3)
    
    
    HibCFG = ReadMax("HibCfg")[1]
    WriteMax(0x60, 0x90) # Exit Hibernate Mode step 1
    WriteMax("HibCfg", 0x0) # Exit Hibernate Mode step 2
    WriteMax(0x60, 0x0) # Exit Hibernate Mode step 3

    WriteMax("DesignCap", DesignCap) # Write DesignCap
    WriteMax("IChgTerm", IchgTerm) # Write IchgTerm
    WriteMax("VEmpty", VEmpty) # Write VEmpty
    if (ChargeVoltage > 4.275):
        WriteMax("ModelCFG", 0x8400) # Write ModelCFG
    else:
        WriteMax("ModelCFG", 0x8000) # Write ModelCFG
    
    #Poll ModelCFG.Refresh(highest bit),
    #proceed to Step 3 when ModelCFG.Refresh=0.
    while (ReadMax("Refresh")[1]):
        time.sleep(10 / 1e3)

    WriteMax("HibCfg", HibCFG) # Restore Original HibCFG value

    # Set Temperature measurement to thermistor
    WriteMax("TSel", 1)

    # Clear POR Bit
    WriteMax("POR", 0)

In [ ]:
import sys


def twos(val_str, bytes):
    val_str = val_str.replace(', ', '')
    val = int(val_str, 2)
    b = val.to_bytes(bytes, byteorder=sys.byteorder, signed=False)                                                          
    return int.from_bytes(b, byteorder=sys.byteorder, signed=True)

In [ ]:
# Step 3.2: Read the RepCap and RepSOC Registers

print(ReadMax(0x06)[1] * per_per_lsb)  # RepSOC - Percent
print(ReadMax(0x09)[1] * 78.125 * 1e-6)  # Battery Voltage 
print(twos(ReadMax('Current')[0], 2) / 1e3 * ua_per_lsb)  # Current mA
print(twos(ReadMax('AvgCurrent')[0], 2) / 1e3 * ua_per_lsb)  # Average Current mA

In [ ]:
print(ReadMax(0x05)[1] * mah_per_lsb)  # RepCap - mAh
print(ReadMax(0x11)[1] * sec_per_lsb / 60 / 60)  # TTE  - Sec but i'm converting to hours - time to empty 
print(ReadMax(0x20)[1] * sec_per_lsb / 60 / 60)  # TTF  - Sec but i'm converting to hours - time to full (while charging) 

## Check Status

In [ ]:
# Reference
# https:#pdfserv.maximintegrated.com/en/an/ug6595-modelgauge-m5-host-side-software-implementation-guide.pdf

In [ ]:
time_array = np.array(time_array)
capacity = np.array(capacity)
percent_level = np.array(percent_level)
tte = np.array(tte)
ttg = np.array(ttg)

time_processed = time_array - time_array[0]

plt.figure(figsize=(15, 6))

plt.subplot(2, 2, 1)
plt.plot(np.delete(time_processed, idxs) / 60 / 60, np.delete(capacity, idxs))
plt.grid(True)
plt.title('Capacity')
plt.xlabel('Time [hours]')
plt.ylabel('Capacity [mAh]')

plt.subplot(2, 2, 2)
plt.plot(np.delete(time_processed, idxs) / 60 / 60, np.delete(percent_level, idxs))
plt.grid(True)
plt.title('Percent Level')
plt.xlabel('Time [hours]')
plt.ylabel('Battery Level (%)')

plt.subplot(2, 2, 3)
plt.plot(np.delete(time_processed, idxs) / 60 / 60, np.delete(tte, idxs) / 60 / 60)
plt.grid(True)
plt.title('Time Till Empty')
plt.xlabel('Time [hours]')
plt.ylabel('Hours')

plt.subplot(2, 2, 4)
plt.plot(np.delete(time_processed, idxs) / 60 / 60, np.delete(ttg, idxs) / 60 / 60)
plt.grid(True)
plt.title('Time Till Full')
plt.xlabel('Time [hours]')
plt.ylabel('Hours')

plt.tight_layout()

# plt.savefig('Battery_Check_WeatherStick3.png', dpi=200)